In [16]:
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from scipy.stats import linregress
from sklearn.model_selection import LeaveOneOut
from numbers import Number
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

1. Download the dataset "House price". Price is the target.

In [2]:
data = pd.read_csv('archive/House_Price.csv')

In [3]:
data = data.sample(frac=1, random_state=34).reset_index(drop=True)

In [4]:
#We check and see that all rows have some missing data
count_rows_with_question_mark = (data == '?').any(axis=1).sum()
count_rows_with_question_mark

np.int64(0)

In [5]:
data.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,dist2,dist3,dist4,teachers,poor_prop,airport,n_hos_beds,n_hot_rooms,waterbody,rainfall,bus_ter,parks
0,21.5,0.11069,43.89,0.550,5.951,93.8,2.92,2.88,3.08,2.68,23.6,17.92,YES,8.730,14.1720,NaN,49,YES,0.048032
1,50.0,6.53876,48.10,0.631,7.016,97.5,1.40,0.92,1.20,1.28,19.8,2.96,YES,10.100,12.4000,NaN,46,YES,0.059229
2,15.2,5.44114,48.10,0.713,6.655,98.2,2.51,2.23,2.65,2.03,19.8,17.73,YES,8.104,11.1216,NaN,41,YES,0.061386
3,24.3,0.53700,36.20,0.504,5.981,68.1,3.97,3.38,3.88,3.47,22.6,11.65,NO,8.286,10.1944,NaN,59,YES,0.046602
4,25.0,2.92400,49.58,0.605,6.101,93.0,2.37,2.28,2.59,1.89,25.3,9.81,YES,10.000,12.2000,River,22,YES,0.053283


In [ ]:


data['airport'] = data['airport'].replace({'YES': 1, 'NO': 0}).infer_objects(copy=False)
data['lake'] = data['waterbody'].isin(['Lake', 'Lake and River']).astype(int).infer_objects(copy=False)
data['river'] = data['waterbody'].isin(['River', 'Lake and River']).astype(int).infer_objects(copy=False)
data['bus_ter'] = data['bus_ter'].replace({'YES': 1, 'NO': 0}).infer_objects(copy=False)

data.drop(columns=['airport', 'waterbody', 'bus_ter'], inplace=True)

In [7]:
data = data.dropna()

X = data.drop(columns=['price'])
y = data['price']

split_ratio = 0.7
split_index = int(len(data) * split_ratio) 
train_X = X.iloc[:split_index].to_numpy()
test_X = X.iloc[split_index:].to_numpy()

train_y = y.iloc[:split_index].to_numpy()
test_y = y.iloc[split_index:].to_numpy()

print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(348, 17)
(150, 17)
(348,)
(150,)


2. Implement the regression tree algorithm from scratch.

3. Select a criteria of your choice to stop splitting the nodes.

```if (depth >= self.max_depth or n_samples < self.min_samples_split):```

In [33]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  
        self.n_features = None

    def is_leaf_node(self):
        return self.value is not None

    
class RegressionTree:
    def __init__(self, min_samples_split=2, max_depth=2, max_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_features = max_features
        self.n_features = None
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.root = self.grow_tree(X, y)

    def best_split(self, X, y):
        n_samples, n_features = X.shape
        if self.max_features:
            features = np.random.choice(n_features, self.max_features, replace=False)
        else:
            features = range(n_features)

        best_mse = float('inf')
        split_idx, split_thresh = None, None
        for feature in features:
            X_column = X[:, feature]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                mse = self.calculate_mse(X_column, y, threshold)
                if mse < best_mse:
                    best_mse = mse
                    split_idx = feature
                    split_thresh = threshold
        return split_idx, split_thresh

    def grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]

        if (depth >= self.max_depth or n_samples < self.min_samples_split):
            return Node(value=np.mean(y))

        feat_idx, threshold = self.best_split(X, y)
        if feat_idx is None:
            return Node(value=np.mean(y))

        left_idxs = X[:, feat_idx] < threshold
        right_idxs = X[:, feat_idx] >= threshold
        left_child = self.grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_child = self.grow_tree(X[right_idxs], y[right_idxs], depth + 1)
        return Node(feature=feat_idx, threshold=threshold, left=left_child, right=right_child)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.root
        while node.left:
            if inputs[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

4. Build a regression tree for the selected dataset.

In [9]:
tree = RegressionTree(min_samples_split=10, max_depth=5)
tree.fit(train_X, train_y)

In [10]:
y_pred = tree.predict(test_X)

In [11]:
mse = mean_squared_error(test_y, y_pred)
mae = mean_absolute_error(test_y, y_pred)
r2 = r2_score(test_y, y_pred)

In [12]:
print("Metric results \nMSE = {}\nMAE = {}\nR^2 = {}".format(mse, mae, r2))

Metric results 
MSE = 19.304885168032
MAE = 3.0354602651734233
R^2 = 0.7117784814017458


5. Test the regressing three using cross-validation. Test different stopping criteria

In [ ]:


splits = [2,4,5,10,20,25,30,35,40]  
depths = [1,2,4,5,10,20,25,30,35,40]
best_r2 = 0
best_split = 0
best_depth = 0

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for split in splits:
    for depth in depths:
        r2_scores = []
        for train_index, val_index in kf.split(train_X):
            X_train, X_val = train_X[train_index], train_X[val_index]
            y_train, y_val = train_y[train_index], train_y[val_index]
            
            tree = RegressionTree(min_samples_split=split, max_depth=depth)
            tree.fit(X_train, y_train)
            y_pred = tree.predict(X_val)
            r2 = r2_score(y_val, y_pred)
            r2_scores.append(r2)
        
        avg_r2 = np.mean(r2_scores)
        if avg_r2 > best_r2:
            best_r2 = avg_r2
            best_split = split
            best_depth = depth

print(f"Best R² = {best_r2}\nBest split = {best_split}\nBest depth = {best_depth}")

Best R² = 0.7790002927334596
Best split = 30
Best depth = 10


In [17]:


sklearn_tree = DecisionTreeRegressor(min_samples_split=10, max_depth=5)
sklearn_cv_scores = cross_val_score(sklearn_tree, X, y, cv=5, scoring='neg_mean_squared_error')
sklearn_cv_mean_score = -np.mean(sklearn_cv_scores)
print(f'Scikit-Learn Cross-Validation Mean Squared Error: {sklearn_cv_mean_score}')

Scikit-Learn Cross-Validation Mean Squared Error: 18.53730337767283


6. Compare the cross-validation results with those you get while building a regression tree with scikit-learn. Use the same cross-validation splits on both models


```Scikit learn -> MSE = 18.54```

```Lastna implementacija -> MSE = 19.3```

Razlika je zaradi globine.  V lastni implementaciji  pridemo samo do globine 5.



7. BONUS (+ 2 points)
Modify your regression tree algorithm to perform as regression trees in random forest. In random forest trees in each split, only a portion of randomly chosen features is considered.

Mal sem rabil tudi regression tree algoritm spremenit


In [34]:
class RandomForestRegressor:
    def __init__(self, n_estimators=100, min_samples_split=2, max_depth=2, max_features=None):
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample = X[indices]
            y_sample = y[indices]
            
            tree = RegressionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                max_features=self.max_features
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Average 
        return np.mean(predictions, axis=0)

In [ ]:

rf_regressor = RandomForestRegressor(
    n_estimators=100,
    min_samples_split=10,
    max_depth=5,
    max_features=5
)


X_train_np = X_train.values if hasattr(X_train, 'values') else X_train
y_train_np = y_train.values if hasattr(y_train, 'values') else y_train
X_test_np = test_X.values if hasattr(test_X, 'values') else test_X
y_test_np = test_y.values if hasattr(test_y, 'values') else test_y


rf_reg_tree = RandomForestRegressionTree(min_samples_split=10, max_depth=5, max_features=5)


rf_reg_tree.fit(X_train_np, y_train_np)


y_pred_rf = rf_reg_tree.predict(X_test_np)
mse_rf = np.mean((y_test_np - y_pred_rf) ** 2)
print(f'Random Forest Regression Tree Mean Squared Error: {mse_rf}')

Random Forest Regression Tree Mean Squared Error: 18.526284262492513


```Lastna implementacija -> MSE = 19.3```

```Scikit learn -> MSE = 18.54```

```Random forest -> MSE = 18.526```